# DLMI - Lymphocytosis classification
## One-to-one image model: CNN

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics, model_selection
from torch.utils.data import DataLoader
from utils_pretrains import *

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
data_dir = "../../data/dlmi-lymphocytosis-classification/"
train_dir = data_dir + "trainset/"
test_dir = data_dir + "testset/"

In [4]:
train_val_images = load_images(train_dir)
test_images = load_images(test_dir)

In [5]:
trainset_true_df = pd.read_csv(data_dir + "trainset/trainset_true.csv")
labels_dict = dict(zip(trainset_true_df["ID"], trainset_true_df["LABEL"]))

In [6]:
id_train_val = list(train_val_images.keys())
id_train, id_val = model_selection.train_test_split(id_train_val, test_size=0.25)

train_images = {id: train_val_images[id] for id in id_train}
val_images = {id: train_val_images[id] for id in id_val}

X_train, y_train = assign_labels(train_images, labels_dict)
X_val, y_val = assign_labels(val_images, labels_dict)
len(X_train), len(X_val)

(9935, 3518)

In [7]:
transform_function = lambda x: x.reshape(3, x.shape[0], x.shape[1])

In [8]:
batch_size = 32
train_dataset = ImageLabelDataset(X_train, y_train, transform_function)
test_dataset = ImageLabelDataset(X_val, y_val, transform_function)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class Classifier(nn.Module):

    def __init__(
        self,
        d_model=2048,
        num_classes=2
    ):
        super().__init__()
        self.d_model = d_model
        self.head = nn.Sequential(
            nn.Linear(d_model, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.head(x)
        return x

In [10]:
image_encoder = Resnet18Encoder()
model = Classifier(d_model=).to(device)
lr = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=lr) 
loss_fn = nn.CrossEntropyLoss()

n_epochs = 50
trainer(
    model, image_encoder, optimizer, loss_fn, train_dataloader, test_dataloader, n_epochs,
    device=device, verbose=True, verbose_every=1, plot=True
)

[Epoch 1 / 50]
	train loss = 0.95583 acc = 0.78822 bacc = 0.50031
	test loss = 0.40020 acc = 0.83513 bacc = 0.50000
[Epoch 2 / 50]
	train loss = 0.47758 acc = 0.79748 bacc = 0.50000
	test loss = 0.40603 acc = 0.83513 bacc = 0.50000
[Epoch 3 / 50]
	train loss = 0.47718 acc = 0.79748 bacc = 0.50000
	test loss = 0.49451 acc = 0.83513 bacc = 0.50000
[Epoch 4 / 50]
	train loss = 0.46058 acc = 0.79748 bacc = 0.50000
	test loss = 0.45210 acc = 0.83513 bacc = 0.50000
[Epoch 5 / 50]
	train loss = 0.44143 acc = 0.79748 bacc = 0.50000
	test loss = 0.49900 acc = 0.83513 bacc = 0.50000
[Epoch 6 / 50]
	train loss = 0.43615 acc = 0.79748 bacc = 0.50000
	test loss = 0.37016 acc = 0.83513 bacc = 0.50000
[Epoch 7 / 50]
	train loss = 0.42279 acc = 0.79889 bacc = 0.54297
	test loss = 0.44898 acc = 0.83513 bacc = 0.50000
[Epoch 8 / 50]
	train loss = 0.40118 acc = 0.81470 bacc = 0.61962
	test loss = 0.41159 acc = 0.79960 bacc = 0.76586
[Epoch 9 / 50]
	train loss = 0.39600 acc = 0.82426 bacc = 0.66288
	test 

In [ ]:
y_true_val = np.array([labels_dict[p] for p in val_images.keys()])
y_pred_min = predict_all(
    model, val_images, transform_function, batch_size=batch_size, 
    prediction_aggregator=PredictionAggregator.MIN, device=device
)
y_pred_max = predict_all(
    model, val_images, transform_function, batch_size=batch_size, 
    prediction_aggregator=PredictionAggregator.MAX, device=device
)
y_pred_mean = predict_all(
    model, val_images, transform_function, batch_size=batch_size, 
    prediction_aggregator=PredictionAggregator.MEAN, device=device
)

In [ ]:
acc = metrics.accuracy_score(y_true_val, y_pred_min)
balanced_acc = metrics.balanced_accuracy_score(y_true_val, y_pred_min)
print(f"Min acc = {acc:.2f} bal. acc = {balanced_acc:.2f}")
acc = metrics.accuracy_score(y_true_val, y_pred_max)
balanced_acc = metrics.balanced_accuracy_score(y_true_val, y_pred_max)
print(f"Max acc = {acc:.2f} bal. acc = {balanced_acc:.2f}")
acc = metrics.accuracy_score(y_true_val, y_pred_mean)
balanced_acc = metrics.balanced_accuracy_score(y_true_val, y_pred_mean)
print(f"Mean acc = {acc:.2f} bal. acc = {balanced_acc:.2f}")

acc = 0.71 bal. acc = 0.50
